# Temporal Difference Learning for the Traveling Salesman Problem

## Overview

This notebook implements **Q-Learning**, a model-free Reinforcement Learning algorithm, to solve the Traveling Salesman Problem (TSP). The approach demonstrates how sequential decision-making algorithms can be applied to combinatorial optimization problems.

## Algorithm: Q-Learning with ε-Greedy Exploration

**Q-Learning Update Rule:**
$$Q(s, a) \leftarrow Q(s, a) + \alpha \left[ r + \gamma \max_{a'} Q(s', a') - Q(s, a) \right]$$

Where:
- $Q(s, a)$: Value of taking action $a$ in state $s$
- $\alpha$: Learning rate (step size)
- $\gamma$: Discount factor for future rewards
- $r$: Immediate reward (negative distance in TSP)

## Key Components

1. **State Space**: Current city and set of visited cities
2. **Action Space**: Unvisited cities to travel to next
3. **Reward Function**: Negative Euclidean distance (minimize total distance)
4. **Exploration Strategy**: ε-greedy (balance exploration vs exploitation)

## Applications in Operations Research

This approach relates to:
- **Vehicle Routing Problems**: Similar sequential decision structure
- **Driver Assignment**: Deciding which driver to dispatch next
- **Delivery Sequencing**: Optimizing the order of deliveries

---

In [ ]:
"""
Temporal Difference (Q-Learning) Algorithm for TSP
================================================

This implementation uses Q-Learning to find near-optimal solutions
to the Traveling Salesman Problem through reinforcement learning.

Author: Sahil Bhatt
"""

import numpy as np
import random
from typing import List, Dict, Tuple


def euclidean_distance(city1: Tuple[float, float], city2: Tuple[float, float]) -> float:
    """Calculate Euclidean distance between two cities."""
    return np.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)


def calculate_path_cost(path: List[int], graph: List[Tuple[float, float]]) -> float:
    """
    Calculate total tour cost (distance) for a given path.
    
    Args:
        path: Ordered list of city indices
        graph: List of city coordinates
        
    Returns:
        Total distance of the tour (including return to start)
    """
    cost = 0
    for i in range(1, len(path)):
        cost += euclidean_distance(graph[path[i]], graph[path[i-1]])
    # Add return to depot
    cost += euclidean_distance(graph[path[-1]], graph[path[0]])
    return cost


def get_next_city(current: int, visited: List[bool], q_values: np.ndarray, 
                  epsilon: float) -> int:
    """
    Select next city using ε-greedy policy.
    
    With probability ε: exploit (choose best Q-value)
    With probability 1-ε: explore (random unvisited city)
    
    Args:
        current: Current city index
        visited: Boolean array of visited cities
        q_values: Q-table [from_city, to_city]
        epsilon: Exploration-exploitation parameter
        
    Returns:
        Index of next city to visit
    """
    candidates = [i for i in range(len(visited)) 
                  if not visited[i] and i != current]
    
    if np.random.random() < epsilon:
        # Exploit: choose city with highest Q-value
        q_vals = [q_values[current, c] for c in candidates]
        return candidates[np.argmax(q_vals)]
    else:
        # Explore: random choice
        return random.choice(candidates)


def q_learning_tsp(graph: List[Tuple[float, float]], 
                   num_episodes: int = 1000,
                   epsilon: float = 0.9,
                   learning_rate: float = 0.1,
                   discount_factor: float = 0.9,
                   verbose: bool = False) -> Tuple[List[int], float, np.ndarray]:
    """
    Train Q-Learning agent to solve TSP.
    
    Args:
        graph: List of city coordinates (x, y)
        num_episodes: Number of training episodes
        epsilon: Probability of exploitation (vs exploration)
        learning_rate: Q-value update step size (α)
        discount_factor: Future reward discount (γ)
        verbose: Print Q-values during training
        
    Returns:
        Tuple of (optimal_path, path_cost, q_values)
    """
    n_cities = len(graph)
    depot = 0  # Starting city
    
    # Initialize Q-table
    q_values = np.zeros((n_cities, n_cities))
    
    # Training loop
    for episode in range(num_episodes):
        current = depot
        visited = [False] * n_cities
        visited[depot] = True
        cities_visited = 1
        
        while cities_visited < n_cities:
            # Select next city
            old_city = current
            current = get_next_city(current, visited, q_values, epsilon)
            
            # Calculate reward (negative distance)
            reward = -euclidean_distance(graph[old_city], graph[current])
            
            # Mark as visited
            visited[current] = True
            cities_visited += 1
            
            # Q-Learning update
            old_q = q_values[old_city, current]
            
            if cities_visited == n_cities:
                # Terminal state: consider return to depot
                q_values[old_city, current] += learning_rate * (
                    discount_factor * q_values[current, depot] - old_q
                )
            else:
                # Non-terminal: bootstrap from max Q-value
                unvisited = [i for i in range(n_cities) 
                            if not visited[i] and i != current]
                max_future_q = max(q_values[current, c] for c in unvisited)
                
                td_error = reward + discount_factor * max_future_q - old_q
                q_values[old_city, current] += learning_rate * td_error
        
        if verbose and episode % 100 == 0:
            print(f"Episode {episode}: Training...")
    
    # Extract optimal path from learned Q-values
    optimal_path = [depot]
    visited = [False] * n_cities
    visited[depot] = True
    
    for _ in range(n_cities - 1):
        current = optimal_path[-1]
        candidates = [i for i in range(n_cities) 
                     if not visited[i] and i != current]
        q_vals = [q_values[current, c] for c in candidates]
        next_city = candidates[np.argmax(q_vals)]
        optimal_path.append(next_city)
        visited[next_city] = True
    
    path_cost = calculate_path_cost(optimal_path, graph)
    
    return optimal_path, path_cost, q_values


# =============================================================================
# MAIN EXECUTION
# =============================================================================

if __name__ == '__main__':
    # Test problem: 5 cities
    cities = [(1, 2), (2, 3), (4, 5), (14, 1), (8, 6)]
    
    print("=" * 50)
    print("Q-LEARNING FOR TRAVELING SALESMAN PROBLEM")
    print("=" * 50)
    print(f"\nCities: {cities}")
    print(f"Number of cities: {len(cities)}")
    
    # Train Q-Learning agent
    optimal_path, cost, q_table = q_learning_tsp(
        graph=cities,
        num_episodes=1000,
        epsilon=0.9,
        learning_rate=0.1,
        discount_factor=0.9
    )
    
    print(f"\n{'='*50}")
    print("RESULTS")
    print(f"{'='*50}")
    print(f"Optimal path: {optimal_path}")
    print(f"Total distance: {cost:.4f}")
    print(f"\nLearned Q-values:\n{q_table}")

q_values:  [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
q_values:  [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
q_values:  [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
q_values:  [[ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0 -1  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
q_values:  [[ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0 -1  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
q_values:  [[ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0 -1  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
q_values:  [[ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0 -1  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
q_values:  [[ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0 -1  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
q_values:  [[ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0 -1  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
q_values:  [[ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0 -1  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
q_values:  [[ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0 

## Conclusion

The Q-Learning algorithm successfully learns a policy for solving small-scale TSP instances. While this approach doesn't scale to very large problems due to the state space explosion, it demonstrates the fundamental principles of:

1. **State-action value estimation** through temporal difference learning
2. **Exploration-exploitation trade-off** via ε-greedy policy
3. **Bootstrap learning** from estimated future values

### Extensions for Production Systems

For larger-scale deployment, consider:
- **Deep Q-Networks (DQN)**: Neural network function approximation
- **Policy Gradient Methods**: Direct policy optimization (PPO, A2C)
- **Hybrid Approaches**: Combining RL with classical OR methods

---

*This notebook is part of a research project on optimization algorithms for crowdsourced delivery platforms.*